In [13]:
!pip install transformers datasets evaluate accelerate -q

In [14]:
import os
os.environ["HF_ALLOW_CODE_EVAL"] = "1"
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [15]:
from datasets import load_dataset
from evaluate import load
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from tqdm import tqdm

In [16]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [17]:
human_eval = load_dataset("openai_humaneval")['test']

In [18]:
code_eval_metric = load("code_eval")

In [19]:
# model_id = "TinyLlama/TinyLlama_v1.1"
model_id = "codeparrot/codeparrot-small"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.float16
)

model.eval()

tokenizer_config.json:   0%|          | 0.00/259 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/903 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/457M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/457M [00:00<?, ?B/s]

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(32768, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=32768, bias=False)
)

In [20]:

# Fix special tokens
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '<pad>'})
if tokenizer.eos_token is None:
    tokenizer.add_special_tokens({'eos_token': '</s>'})

# Resize embeddings if new special tokens added
if len(tokenizer) > model.config.vocab_size:
    model.resize_token_embeddings(len(tokenizer))

In [43]:
num_samples_per_problem = 20
test_cases = []
candidates = []

In [44]:
for problem in tqdm(human_eval.select(range(10)), desc="Problems", unit="problem"):  # test on 10 problems
    prompt = problem['prompt']
    test_code = problem['test']
    test_cases.append(test_code)
    problem_candidates = []

    for _ in range(num_samples_per_problem):
        inputs = tokenizer(prompt, return_tensors="pt").to(device)

        with torch.no_grad():
            outputs = model.generate(
                input_ids=inputs['input_ids'],
                attention_mask=inputs['attention_mask'],
                max_length=512,
                do_sample=True,
                temperature=0.7,
                top_p=0.95,
                num_return_sequences=1,
                pad_token_id=tokenizer.pad_token_id,
                eos_token_id=tokenizer.eos_token_id,
            )
        generated_code = tokenizer.decode(outputs[0], skip_special_tokens=True)
        generated_code = generated_code[len(prompt):]
        problem_candidates.append(generated_code)

    candidates.append(problem_candidates)

Problems: 100%|██████████| 10/10 [09:20<00:00, 56.05s/problem]


In [45]:
len(candidates)

10

In [46]:
print("Evaluating generated code...")
k_values = [1, 5, 10, 20]
pass_at_k, results = code_eval_metric.compute(
    references=test_cases,
    predictions=candidates,
    k=k_values,
    num_workers=4,
    timeout=10.0
)

Evaluating generated code...


In [47]:
for k in k_values:
    print(f"Pass@{k}: {pass_at_k[f'pass@{k}'] * 100:.2f}%")

Pass@1: 39.50%
Pass@5: 89.88%
Pass@10: 98.60%
Pass@20: 100.00%


In [48]:
model_id = "TinyLlama/TinyLlama_v1.1"
# model_id = "codeparrot/codeparrot-small"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.float16
)

model.eval()

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 2048)
    (layers): ModuleList(
      (0-21): 22 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=5632, bias=False)
          (up_proj): Linear(in_features=2048, out_features=5632, bias=False)
          (down_proj): Linear(in_features=5632, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((2048,), eps=1e-05)
    (rotary_emb): 

In [49]:
# Fix special tokens
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '<pad>'})
if tokenizer.eos_token is None:
    tokenizer.add_special_tokens({'eos_token': '</s>'})

# Resize embeddings if new special tokens added
if len(tokenizer) > model.config.vocab_size:
    model.resize_token_embeddings(len(tokenizer))

num_samples_per_problem = 20
test_cases = []
candidates = []

for problem in tqdm(human_eval.select(range(10)), desc="Problems", unit="problem"):  # test on 10 problems
    prompt = problem['prompt']
    test_code = problem['test']
    test_cases.append(test_code)
    problem_candidates = []

    for _ in range(num_samples_per_problem):
        inputs = tokenizer(prompt, return_tensors="pt").to(device)

        with torch.no_grad():
            outputs = model.generate(
                input_ids=inputs['input_ids'],
                attention_mask=inputs['attention_mask'],
                max_length=512,
                do_sample=True,
                temperature=0.7,
                top_p=0.95,
                num_return_sequences=1,
                pad_token_id=tokenizer.pad_token_id,
                eos_token_id=tokenizer.eos_token_id,
            )
        generated_code = tokenizer.decode(outputs[0], skip_special_tokens=True)
        generated_code = generated_code[len(prompt):]
        problem_candidates.append(generated_code)

    candidates.append(problem_candidates)



Problems: 100%|██████████| 10/10 [44:50<00:00, 269.01s/problem]


In [50]:
print("Evaluating generated code...")
k_values = [1, 5, 10, 20]
pass_at_k2, results2 = code_eval_metric.compute(
    references=test_cases,
    predictions=candidates,
    k=k_values,
    num_workers=4,
    timeout=10.0
)

print(pass_at_k2)

for k in k_values:
    print(f"Pass@{k}: {pass_at_k2[f'pass@{k}'] * 100:.2f}%")

Evaluating generated code...
{'pass@1': np.float64(0.009999999999999998), 'pass@5': np.float64(0.04473684210526315), 'pass@10': np.float64(0.0763157894736842), 'pass@20': np.float64(0.1)}
Pass@1: 1.00%
Pass@5: 4.47%
Pass@10: 7.63%
Pass@20: 10.00%


In [51]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

In [52]:
model = prepare_model_for_kbit_training(model)

peft_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "v_proj"]
)
model = get_peft_model(model, peft_config)


In [53]:
model.print_trainable_parameters()

trainable params: 1,126,400 || all params: 1,101,174,784 || trainable%: 0.1023


In [54]:
ds = load_dataset("openai_humaneval")

In [55]:
print(ds)

DatasetDict({
    test: Dataset({
        features: ['task_id', 'prompt', 'canonical_solution', 'test', 'entry_point'],
        num_rows: 164
    })
})


Okay, so code parrot works kinda at pass@5, 100%. Whereas that's not case with tinyllama. Lets see if i can finetune it to make this better.